In [1]:
import numpy
import pandas as pd
import numpy as np
import scipy.spatial
import tensorflow as tf 
import sklearn.naive_bayes

# Load data  

In [2]:
train_tweets = pd.read_csv('/Users/juliachen/Desktop/DSCI552/hw/hw5/data/ps5_tweets_text.csv')
train_label = pd.read_csv('/Users/juliachen/Desktop/DSCI552/hw/hw5/data/ps5_tweets_labels_as_numbers.csv')

test_tweets = pd.read_csv('/Users/juliachen/Desktop/DSCI552/hw/hw5/data/ps5_tweets_text_for_the_kaggle_competition.csv')

In [3]:
print(f"Train data shape {train_tweets.shape}")
print(f"Test data shape {test_tweets.shape}")

Train data shape (37041, 2)
Test data shape (3798, 2)


In [4]:
#'''delete id column'''
train_tweets.drop("Id",axis=1,inplace=True)
train_label.drop("Id",axis=1,inplace=True)

'''Combine train_tweets and train_label into train data set'''
train = pd.concat([train_label, train_tweets], axis=1)

#train.to_csv("/Users/juliachen/Desktop/DSCI552/hw/hw5/data/train.csv")
#train = pd.read_csv('/Users/juliachen/Desktop/DSCI552/hw/hw5/data/train.csv')
#train = train.drop(columns=['Unnamed: 0'])

In [5]:
train.head()

,Label,Tweet
0,4,https://t.co/UpjxfOgQs8\r\r\n\r\r\nGaisss! Ple...
1,1,@mygovindia Today just after a week of lockdow...
2,2,Tuskys partners with Amref to provide on groun...
3,1,@chrissyteigen are u doing ur own grocery shop...
4,0,UK Critical Care Nurse Cries at Empty SuperMar...


In [6]:
train['Label'].value_counts()

3    10282
1     8930
2     6930
4     5953
0     4946
Name: Label, dtype: int64

In [7]:
train_tweets.head()

,Tweet
0,https://t.co/UpjxfOgQs8\r\r\n\r\r\nGaisss! Ple...
1,@mygovindia Today just after a week of lockdow...
2,Tuskys partners with Amref to provide on groun...
3,@chrissyteigen are u doing ur own grocery shop...
4,UK Critical Care Nurse Cries at Empty SuperMar...


In [8]:
train_tweets['Tweet']

0        https://t.co/UpjxfOgQs8\r\r\n\r\r\nGaisss! Ple...
1        @mygovindia Today just after a week of lockdow...
2        Tuskys partners with Amref to provide on groun...
3        @chrissyteigen are u doing ur own grocery shop...
4        UK Critical Care Nurse Cries at Empty SuperMar...
                               ...                        
37036    Minnesota classifies grocery store workers as ...
37037    US Senator @ewarren has asked for information ...
37038    Just commented on @thejournal_ie: Poll: Are yo...
37039    My wife got laid off yesterday because the sma...
37040    Humanity is doomed\r\r\n#coronavirus #coronacr...
Name: Tweet, Length: 37041, dtype: object

# Tokenizer

The process of tokenization is to represent words in a way that computer can understand and be able to train it. 

num_words: the maximum number of words to keep, based on word frequency.

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer

#tokenizer = Tokenizer(num_words=10000)
                      #filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                      #lower=True,
                      #split=" ")

            
tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>',
                     filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                     lower=True)
            
            
tokenizer.fit_on_texts(train_tweets['Tweet'])

word_index= tokenizer.word_index

In [10]:
one_hot_train = tokenizer.texts_to_matrix(train_tweets['Tweet'], mode='binary')

In [11]:
'''Split training and validation set'''

"""Create random indices"""
np.random.seed(42)
N = len(train_tweets)
shuffled_indices = np.random.permutation(N)
train_label=train_label.to_numpy()

"""Split indices into train and val set"""
train_indices = shuffled_indices[:int(N*0.8)]
valid_indices = shuffled_indices[int(N*0.8):]

"""Spilt"""
training_sentences = one_hot_train[train_indices]
training_labels = train_label[train_indices]

valid_sentences = one_hot_train[valid_indices]
valid_labels = train_label[valid_indices]

In [12]:
train_indices

array([11836, 35248, 23784, ..., 27958,  8399,  9079])

In [13]:
print('The Shape of training x',training_sentences.shape)
print('The Shape of testing x',valid_sentences.shape)

print('The Shape of training y',training_labels.shape)
print('The Shape of testing y',valid_labels.shape)

The Shape of training x (29632, 5000)
The Shape of testing x (7409, 5000)
The Shape of training y (29632, 1)
The Shape of testing y (7409, 1)


In [14]:
train_ExN = np.array(valid_labels==0).astype('int')
train_Neg = np.array(valid_labels==1).astype('int')
train_Neu = np.array(valid_labels==2).astype('int')
train_Pos = np.array(valid_labels==3).astype('int')
train_ExP = np.array(valid_labels==4).astype('int')

# Baseline Model - Naive Bayes

In [15]:
bayes = sklearn.naive_bayes.MultinomialNB()
bayes.fit(training_sentences, training_labels.ravel())

MultinomialNB()

In [16]:
prediction1 = bayes.predict(valid_sentences)
prediction1

array([1, 2, 1, ..., 3, 2, 1])

In [17]:
from sklearn import metrics
print('Accuracy: ',metrics.accuracy_score(prediction1, valid_labels))
#print('F1 Score:', metrics.f1_score(prediction1, valid_labels, average='weighted'))

print(metrics.classification_report(prediction1, valid_labels))

Accuracy:  0.48616547442299907
              precision    recall  f1-score   support

           0       0.50      0.48      0.49       991
           1       0.43      0.46      0.44      1755
           2       0.65      0.53      0.58      1645
           3       0.40      0.45      0.42      1810
           4       0.54      0.54      0.54      1208

    accuracy                           0.49      7409
   macro avg       0.50      0.49      0.49      7409
weighted avg       0.50      0.49      0.49      7409



In [18]:
from sklearn.metrics import f1_score
print("Micro F1 score: ",f1_score(valid_labels, prediction1, average='micro'))
print("Macro F1 score: ",f1_score(valid_labels, prediction1, average='macro'))

Micro F1 score:  0.48616547442299907
Macro F1 score:  0.49425341334830347
